In [6]:
# !pip install --quiet yfinance prophet

import pandas as pd
from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import logging
logging.getLogger('prophet').setLevel(logging.ERROR) # Suppress Prophet verbose output

# Load the data
df = pd.read_csv('/content/Nvidia_stock_data.csv')

# Rename columns for Prophet: 'ds' for date, 'y' for the value to predict
df['ds'] = pd.to_datetime(df['Date'])

# Prepare data for Prophet for each target variable
df_volume = df[['ds', 'Volume']].copy()
df_volume.rename(columns={'Volume': 'y'}, inplace=True)

df_high = df[['ds', 'High']].copy()
df_high.rename(columns={'High': 'y'}, inplace=True)

df_low = df[['ds', 'Low']].copy()
df_low.rename(columns={'Low': 'y'}, inplace=True)

# Function to train Prophet model and make predictions
def predict_prophet(df_data, future_periods):
    model = Prophet()
    model.fit(df_data)
    future = model.make_future_dataframe(periods=future_periods, freq='D')
    forecast = model.predict(future)
    return forecast

# Define the date to start prediction from (July 1, 2025)
start_date = pd.to_datetime('2025-07-01')

# Filter the data to only include data before the prediction start date for training
df_volume_train = df_volume[df_volume['ds'] < start_date]
df_high_train = df_high[df_high['ds'] < start_date]
df_low_train = df_low[df_low['ds'] < start_date]

# Calculate the number of future periods needed for six months
# This is an approximation, assuming roughly 30.4 days per month
periods_six_months = 6 * 30

# Predict for Volume
forecast_volume = predict_prophet(df_volume_train, periods_six_months)
# Filter for the prediction period
forecast_volume_future = forecast_volume[(forecast_volume['ds'] >= start_date) & (forecast_volume['ds'] < start_date + pd.DateOffset(months=6))].copy()
# Ensure Volume predictions are non-negative and integers
forecast_volume_future['yhat'] = forecast_volume_future['yhat'].apply(lambda x: max(0, x)).round().astype(int)

# Predict for High
forecast_high = predict_prophet(df_high_train, periods_six_months)
# Filter for the prediction period
forecast_high_future = forecast_high[(forecast_high['ds'] >= start_date) & (forecast_high['ds'] < start_date + pd.DateOffset(months=6))].copy()
# Ensure High predictions are non-negative
forecast_high_future['yhat'] = forecast_high_future['yhat'].apply(lambda x: max(0, x))

# Predict for Low
forecast_low = predict_prophet(df_low_train, periods_six_months)
# Filter for the prediction period
forecast_low_future = forecast_low[(forecast_low['ds'] >= start_date) & (forecast_low['ds'] < start_date + pd.DateOffset(months=6))].copy()
# Ensure Low predictions are non-negative
forecast_low_future['yhat'] = forecast_low_future['yhat'].apply(lambda x: max(0, x))


# Display predictions (first few and last few rows)
print("Predicted Volume for the next six months from July 2025:")
print(forecast_volume_future[['ds', 'yhat']].head())
print("...")
print(forecast_volume_future[['ds', 'yhat']].tail())

print("\nPredicted High for the next six months from July 2025:")
print(forecast_high_future[['ds', 'yhat']].head())
print("...")
print(forecast_high_future[['ds', 'yhat']].tail())

print("\nPredicted Low for the next six months from July 2025:")
print(forecast_low_future[['ds', 'yhat']].head())
print("...")
print(forecast_low_future[['ds', 'yhat']].tail())


merged_forecast = forecast_volume_future[['ds', 'yhat']].rename(columns={'yhat': 'Predicted_Volume'})
merged_forecast['Predicted_High'] = forecast_high_future['yhat'].values
merged_forecast['Predicted_Low'] = forecast_low_future['yhat'].values
print("\nMerged Predictions:")
print(merged_forecast.head())
print("...")
print(merged_forecast.tail())

DEBUG:cmdstanpy:input tempfile: /tmp/tmpn2s7otpz/tmdetaby.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn2s7otpz/ekgnzw09.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=28745', 'data', 'file=/tmp/tmpn2s7otpz/tmdetaby.json', 'init=/tmp/tmpn2s7otpz/ekgnzw09.json', 'output', 'file=/tmp/tmpn2s7otpz/prophet_modelasto3haz/prophet_model-20250625115345.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:53:45 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:53:48 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn2s7otpz/o916wcuk.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn2s7otpz/q8sw1c1g.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

Predicted Volume for the next six months from July 2025:
             ds        yhat
6657 2025-07-01   332878301
6658 2025-07-02   344919045
6659 2025-07-03   358613620
6660 2025-07-04   363838156
6661 2025-07-05  1889396228
...
             ds        yhat
6812 2025-12-03   320681475
6813 2025-12-04   333897227
6814 2025-12-05   338030747
6815 2025-12-06  1861816821
6816 2025-12-07  1856067709

Predicted High for the next six months from July 2025:
             ds        yhat
6657 2025-07-01  110.927162
6658 2025-07-02  110.877480
6659 2025-07-03  111.064035
6660 2025-07-04  111.167718
6661 2025-07-05  111.172647
...
             ds        yhat
6812 2025-12-03  120.410765
6813 2025-12-04  120.536497
6814 2025-12-05  120.575015
6815 2025-12-06  120.511961
6816 2025-12-07  120.534917

Predicted Low for the next six months from July 2025:
             ds        yhat
6657 2025-07-01  106.206501
6658 2025-07-02  106.159281
6659 2025-07-03  106.335715
6660 2025-07-04  106.474859
6661 2025-07

In [7]:
# Save the merged forecast to a CSV file
merged_forecast.to_csv('nvidia_stock_predictions.csv', index=False)

print("Predicted stock data saved to 'nvidia_stock_predictions.csv'")

Predicted stock data saved to 'nvidia_stock_predictions.csv'
